# Assocation rule mining with MIMIC-IV dataset: Exploring comorbidity diagnoses of major depressive disorder 



Author:<br>

Dominic Nicolas Iseli<br>
DSV - Department of Computer and Systems Sciences<br>
Stockholm University<br>
Stockholm, Sweden<br>
dois2136@student.su.se

## Import necessary modules

In [1]:
import numpy as np
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules

## Admissions dataset

Read admissions dataset from csv. The dataset includes information about a patient’s admission to the hospital. 

In [2]:
admissions = pd.read_csv('admissions.csv')
admissions.head()

,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,marital_status,ethnicity,edregtime,edouttime,hospital_expire_flag
0,14679932,21038362,2139-09-26 14:16:00,2139-09-28 11:30:00,NaN,ELECTIVE,NaN,HOME,Other,ENGLISH,SINGLE,UNKNOWN,NaN,NaN,0
1,15585972,24941086,2123-10-07 23:56:00,2123-10-12 11:22:00,NaN,ELECTIVE,NaN,HOME,Other,ENGLISH,NaN,WHITE,NaN,NaN,0
2,11989120,21965160,2147-01-14 09:00:00,2147-01-17 14:25:00,NaN,ELECTIVE,NaN,HOME,Other,ENGLISH,NaN,UNKNOWN,NaN,NaN,0
3,17817079,24709883,2165-12-27 17:33:00,2165-12-31 21:18:00,NaN,ELECTIVE,NaN,HOME,Other,ENGLISH,NaN,OTHER,NaN,NaN,0
4,15078341,23272159,2122-08-28 08:48:00,2122-08-30 12:32:00,NaN,ELECTIVE,NaN,HOME,Other,ENGLISH,NaN,BLACK/AFRICAN AMERICAN,NaN,NaN,0


Analyse general properties and null values of the admissions dataset

In [3]:
admissions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 523740 entries, 0 to 523739
Data columns (total 15 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   subject_id            523740 non-null  int64 
 1   hadm_id               523740 non-null  int64 
 2   admittime             523740 non-null  object
 3   dischtime             523740 non-null  object
 4   deathtime             9337 non-null    object
 5   admission_type        523740 non-null  object
 6   admission_location    463305 non-null  object
 7   discharge_location    397083 non-null  object
 8   insurance             523740 non-null  object
 9   language              523740 non-null  object
 10  marital_status        457633 non-null  object
 11  ethnicity             523740 non-null  object
 12  edregtime             311504 non-null  object
 13  edouttime             311504 non-null  object
 14  hospital_expire_flag  523740 non-null  int64 
dtypes: int64(3), obje

Transform admittime column to include only year and store in dictionary together with hadm_id, used for calculating the age of the patients later.

In [4]:
admissions['admittime'] =  pd.to_datetime(admissions['admittime'])
admissions['admittime'] = admissions['admittime'].dt.year
admittime_dict = dict(zip(admissions.hadm_id, admissions.admittime.astype(int)))

## Patient dataset

Read patients dataset from csv-file

In [5]:
patients = pd.read_csv('patients.csv')
patients.head()

,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod
0,10000048,F,23,2126,2008 - 2010,NaN
1,10002723,F,0,2128,2017 - 2019,NaN
2,10003939,M,0,2184,2008 - 2010,NaN
3,10004222,M,0,2161,2014 - 2016,NaN
4,10005325,F,0,2154,2011 - 2013,NaN


Analyse general properties and null values of the patient dataset

In [6]:
patients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 382278 entries, 0 to 382277
Data columns (total 6 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   subject_id         382278 non-null  int64 
 1   gender             382278 non-null  object
 2   anchor_age         382278 non-null  int64 
 3   anchor_year        382278 non-null  int64 
 4   anchor_year_group  382278 non-null  object
 5   dod                9509 non-null    object
dtypes: int64(3), object(3)
memory usage: 17.5+ MB


Create gender, anchor_age and anchor_year dictionaries used for mapping later.

In [7]:
gender_dict = dict(zip(patients.subject_id, patients.gender))
anchor_age_dict = dict(zip(patients.subject_id, patients.anchor_age.astype(int)))
anchor_year_dict = dict(zip(patients.subject_id, patients.anchor_year.astype(int)))

Check the unique number of patients

In [8]:
patients['subject_id'].value_counts().sum()

382278

## Diagnoses code dataset

Read dimension table for diagnoses_icd which provides a description/long title of ICD-9/ICD-10 diagnoses.

In [9]:
diagnoses_codes = pd.read_excel('d_icd_diagnoses.xlsx')
diagnoses_codes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 109775 entries, 0 to 109774
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   icd_code     109775 non-null  object
 1   icd_version  109775 non-null  int64 
 2   long_title   109775 non-null  object
dtypes: int64(1), object(2)
memory usage: 2.5+ MB


Include only ICD-10 codes

In [10]:
diagnoses_codes = diagnoses_codes[diagnoses_codes['icd_version'] == 10]
diagnoses_codes.shape

(95109, 3)

Check for duplicates in ICD-code column

In [11]:
duplicates = diagnoses_codes.loc[diagnoses_codes.duplicated(subset=['icd_code'])]
duplicates

,icd_code,icd_version,long_title


Delete icd_version column, as it is not needed anymore

In [12]:
del diagnoses_codes['icd_version']
diagnoses_codes.head()

,icd_code,long_title
9795,B5741,Meningitis in Chagas' disease
9857,B810,Anisakiasis
10057,B811,Intestinal capillariasis
10981,A5489,Other gonococcal infections
11338,B010,Varicella meningitis


Transform ICD diagnoses codes to dictionary to be used for mapping later

In [13]:
diagnoses_codes.set_index('icd_code', inplace=True)
diagnoses_codes = diagnoses_codes.to_dict()
diagnoses_codes = {str(k):str(v) for k,v in diagnoses_codes['long_title'].items()}

## Diagnoses dataset

Read the diagnoses dataset which includes all billed ICD diagnoses for hospitalizations.

In [14]:
diagnoses = pd.read_csv('diagnoses.csv')

Analyse general properties and null values of the diagnoses dataset

In [15]:
diagnoses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5280351 entries, 0 to 5280350
Data columns (total 5 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   subject_id   int64 
 1   hadm_id      int64 
 2   seq_num      int64 
 3   icd_code     object
 4   icd_version  int64 
dtypes: int64(4), object(1)
memory usage: 201.4+ MB


Include only diagnoses with ICD-10 codes

In [16]:
diagnoses = diagnoses[diagnoses['icd_version'] == 10]

In [17]:
diagnoses.head(5)

,subject_id,hadm_id,seq_num,icd_code,icd_version
2670103,18335503,21596415,2,E861,10
2670104,18335503,21596415,5,O99512,10
2670105,18335503,21596415,6,J45909,10
2670106,18335503,21596415,1,O99612,10
2670107,18335503,21596415,7,O99282,10


Add a long title of the diagnoses to the diagnoses dataset

In [18]:
diagnoses['long_title'] = diagnoses.icd_code.map(diagnoses_codes)

Check how many individual patients there are in the diagnoses dataset, as one patient can have several diagnoses.

In [19]:
len(diagnoses['subject_id'].value_counts())

107704

Check how many individual hadm_id's there are in the dataset, which represent a single patient’s admission to the hospital. One patient can have several admissions to the hospital.


In [20]:
len(diagnoses['hadm_id'].value_counts())

185743

Check for missing values in the diagnoses dataset

In [21]:
diagnoses.isna().sum()

subject_id     0
hadm_id        0
seq_num        0
icd_code       0
icd_version    0
long_title     0
dtype: int64

Calculate the age of each patient for each diagnosis.

In [22]:
diagnoses['admittime'] = diagnoses.hadm_id.map(admittime_dict)
diagnoses['anchor_age'] = diagnoses.subject_id.map(anchor_age_dict)
diagnoses['anchor_year'] = diagnoses.subject_id.map(anchor_year_dict)
diagnoses['birth_year'] = diagnoses['anchor_year'] - diagnoses['anchor_age']
diagnoses['age'] = diagnoses['admittime'] - diagnoses['birth_year']
del diagnoses['admittime']
del diagnoses['anchor_age']
del diagnoses['anchor_year']
del diagnoses['birth_year']
diagnoses

,subject_id,hadm_id,seq_num,icd_code,icd_version,long_title,age
2670103,18335503,21596415,2,E861,10,Hypovolemia,33.0
2670104,18335503,21596415,5,O99512,10,Diseases of the respiratory system complicatin...,33.0
2670105,18335503,21596415,6,J45909,10,"Unspecified asthma, uncomplicated",33.0
2670106,18335503,21596415,1,O99612,10,Diseases of the digestive system complicating ...,33.0
2670107,18335503,21596415,7,O99282,10,"Endocrine, nutritional and metabolic diseases ...",33.0
...,...,...,...,...,...,...,...
5280346,13747041,25594844,6,R531,10,Weakness,45.0
5280347,13747041,25594844,8,R0902,10,Hypoxemia,45.0
5280348,13747041,25594844,4,F1120,10,"Opioid dependence, uncomplicated",45.0
5280349,13747041,25594844,2,J189,10,"Pneumonia, unspecified organism",45.0


Calculate the average age of a patient among all hospital stays (hadm_id's) and create dictionary for mapping later.

In [23]:
avg_age = diagnoses.groupby(['subject_id', 'hadm_id'], as_index=False).mean().groupby('subject_id')['age'].mean().astype(int)
avg_age_dict = avg_age.to_dict()

## ICD-codes to be analyzed/of interest

Define which ICD-codes are of interest and store in list.

In [24]:
codes = ['F329']

Create a new dataset from the diagnoses dataset only containing the ICD-codes of intereset as defined above.

In [25]:
depression_diagnoses = diagnoses[diagnoses['icd_code'].isin(codes)]

Check shape of the dataset with the diagnoses of interest.

In [26]:
depression_diagnoses.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30398 entries, 2935125 to 5280321
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   subject_id   30398 non-null  int64  
 1   hadm_id      30398 non-null  int64  
 2   seq_num      30398 non-null  int64  
 3   icd_code     30398 non-null  object 
 4   icd_version  30398 non-null  int64  
 5   long_title   30398 non-null  object 
 6   age          30398 non-null  float64
dtypes: float64(1), int64(4), object(2)
memory usage: 1.9+ MB


In [27]:
depression_diagnoses.head()

,subject_id,hadm_id,seq_num,icd_code,icd_version,long_title,age
2935125,15548420,27667659,3,F329,10,"Major depressive disorder, single episode, uns...",38.0
2952402,18817907,28875074,9,F329,10,"Major depressive disorder, single episode, uns...",30.0
2972814,15066286,23086326,10,F329,10,"Major depressive disorder, single episode, uns...",39.0
2982523,16213188,26834904,3,F329,10,"Major depressive disorder, single episode, uns...",31.0
2984731,11625936,22823980,18,F329,10,"Major depressive disorder, single episode, uns...",84.0


Store list of all patients with depression.

In [28]:
depr_patients_allseq = depression_diagnoses['subject_id'].unique().tolist()

Only inlcude patients that have depression as a primary diagnoses. We only consider prioritites of 1, 2, 3 assigned to the diagnosis.

In [29]:
seq_num = [1,2,3]
depression_diagnoses = depression_diagnoses[depression_diagnoses['seq_num'].isin(seq_num)]
depression_diagnoses

,subject_id,hadm_id,seq_num,icd_code,icd_version,long_title,age
2935125,15548420,27667659,3,F329,10,"Major depressive disorder, single episode, uns...",38.0
2982523,16213188,26834904,3,F329,10,"Major depressive disorder, single episode, uns...",31.0
3016887,17710225,28227252,2,F329,10,"Major depressive disorder, single episode, uns...",57.0
3025484,17754388,25783452,2,F329,10,"Major depressive disorder, single episode, uns...",38.0
3044503,17014101,23889295,2,F329,10,"Major depressive disorder, single episode, uns...",49.0
...,...,...,...,...,...,...,...
5279599,10664440,24192500,1,F329,10,"Major depressive disorder, single episode, uns...",28.0
5279668,16825095,26628718,3,F329,10,"Major depressive disorder, single episode, uns...",21.0
5279677,13192739,27479647,1,F329,10,"Major depressive disorder, single episode, uns...",54.0
5279778,10264646,20347132,3,F329,10,"Major depressive disorder, single episode, uns...",48.0


Check how many individual patients there are in the depression_diagnoses dataset. One patient can have several depression diagnoses.

In [30]:
len(depression_diagnoses['subject_id'].value_counts())

4416

Get unique subject_id's and hadm_id's from depression diagnoses dataset. This will get a list of all patients that have the diagnoses and a list of all the admissions where the disease was diagnosed.

In [31]:
depr_patients = depression_diagnoses['subject_id'].unique().tolist()
depr_patients_hadm_id = depression_diagnoses['hadm_id'].unique().tolist()

Add demographic information to dataset with depression patients by creating a separate dataframe. Create age groups based on age column. Calculate descpritive statistics of depression patients.

In [32]:
depr_patients_df = pd.DataFrame(depr_patients, columns=['subject_id'])
depr_patients_df['gender'] = depr_patients_df.subject_id.map(gender_dict)
depr_patients_df['age'] = depr_patients_df.subject_id.map(avg_age_dict)
depr_patients_df['age_group'] = pd.cut(x=depr_patients_df['age'], bins=[17,30,39,49,59,69,79,120], labels = ['18-29', '30-39', '40-49', '50-59', '60-69', '70-79', '>80'])
depr_patients_df_describe = depr_patients_df.groupby(["age_group"]).describe()
cols = [0,1,2,3,4,5,6,7]
depr_patients_df_describe.drop(depr_patients_df_describe.columns[cols],axis=1,inplace=True)
depr_patients_df_describe.to_excel(r'output\age_experimental_group.xlsx')
depr_patients_df_describe

age                                                   
            count       mean       std   min   25%   50%   75%   max
age_group                                                           
18-29      1657.0  23.059747  3.466636  18.0  20.0  22.0  26.0  30.0
30-39       770.0  34.880519  2.570971  31.0  33.0  35.0  37.0  39.0
40-49       565.0  44.506195  2.926168  40.0  42.0  44.0  47.0  49.0
50-59       699.0  54.404864  2.870765  50.0  52.0  54.0  57.0  59.0
60-69       461.0  63.687636  2.761151  60.0  61.0  63.0  66.0  69.0
70-79       181.0  73.309392  2.501859  70.0  71.0  73.0  75.0  79.0
>80          83.0  86.771084  4.129978  80.0  83.0  86.0  90.5  96.0

In [33]:
depr_patients_df['gender'].value_counts(normalize=True)

F    0.591033
M    0.408967
Name: gender, dtype: float64

In [34]:
depr_patients_df.groupby(["gender", "age_group"]).count()

subject_id   age
gender age_group                  
F      18-29            1009  1009
       30-39             440   440
       40-49             316   316
       50-59             392   392
       60-69             266   266
       70-79             127   127
       >80                60    60
M      18-29             648   648
       30-39             330   330
       40-49             249   249
       50-59             307   307
       60-69             195   195
       70-79              54    54
       >80                23    23

Patients can have other diagnoses than depression. In the next step, we get all other diagnoses for the depression patients from the diagnoses dataset.

In [35]:
diagnoses_depr_patients = diagnoses[diagnoses['subject_id'].isin(depr_patients)]
diagnoses_depr_patients

,subject_id,hadm_id,seq_num,icd_code,icd_version,long_title,age
2901044,11443746,27515077,16,G43109,10,"Migraine with aura, not intractable, without s...",52.0
2901045,11443746,27515077,18,G4700,10,"Insomnia, unspecified",52.0
2901046,11443746,27515077,14,E669,10,"Obesity, unspecified",52.0
2901047,11443746,27515077,20,M549,10,"Dorsalgia, unspecified",52.0
2901048,11443746,27515077,9,I10,10,Essential (primary) hypertension,52.0
...,...,...,...,...,...,...,...
5280068,11012896,24213599,6,Y929,10,Unspecified place or not applicable,22.0
5280184,15662564,26486058,3,G8929,10,Other chronic pain,66.0
5280185,15662564,26486058,4,J449,10,"Chronic obstructive pulmonary disease, unspeci...",66.0
5280186,15662564,26486058,2,M545,10,Low back pain,66.0


Only include the admissions where the patients have been diagnosed with depression. We only want to study the admissions where the diagnoses of interest was diagnosed.

In [36]:
diagnoses_depr_patients = diagnoses_depr_patients[diagnoses_depr_patients['hadm_id'].isin(depr_patients_hadm_id)]
diagnoses_depr_patients

,subject_id,hadm_id,seq_num,icd_code,icd_version,long_title,age
2935120,15548420,27667659,2,Z370,10,Single live birth,38.0
2935121,15548420,27667659,6,O701,10,Second degree perineal laceration during delivery,38.0
2935122,15548420,27667659,5,O770,10,Labor and delivery complicated by meconium in ...,38.0
2935123,15548420,27667659,1,O99344,10,Other mental disorders complicating childbirth,38.0
2935124,15548420,27667659,4,Z3A39,10,39 weeks gestation of pregnancy,38.0
...,...,...,...,...,...,...,...
5280064,11012896,24213599,2,S22061A,10,"Stable burst fracture of T7-T8 vertebra, initi...",22.0
5280065,11012896,24213599,4,M549,10,"Dorsalgia, unspecified",22.0
5280066,11012896,24213599,1,S22081A,10,"Stable burst fracture of T11-T12 vertebra, ini...",22.0
5280067,11012896,24213599,5,W109XXA,10,"Fall (on) (from) unspecified stairs and steps,...",22.0


Create a new column as a combination of the icd_code and icd_code_long_title, which will help in interpretation of results.

In [37]:
diagnoses_depr_patients = diagnoses_depr_patients.copy()
diagnoses_depr_patients['icd_code_long_title'] = diagnoses_depr_patients['icd_code'].astype(str) + ": " + diagnoses_depr_patients['long_title']
diagnoses_depr_patients

,subject_id,hadm_id,seq_num,icd_code,icd_version,long_title,age,icd_code_long_title
2935120,15548420,27667659,2,Z370,10,Single live birth,38.0,Z370: Single live birth
2935121,15548420,27667659,6,O701,10,Second degree perineal laceration during delivery,38.0,O701: Second degree perineal laceration during...
2935122,15548420,27667659,5,O770,10,Labor and delivery complicated by meconium in ...,38.0,O770: Labor and delivery complicated by meconi...
2935123,15548420,27667659,1,O99344,10,Other mental disorders complicating childbirth,38.0,O99344: Other mental disorders complicating ch...
2935124,15548420,27667659,4,Z3A39,10,39 weeks gestation of pregnancy,38.0,Z3A39: 39 weeks gestation of pregnancy
...,...,...,...,...,...,...,...,...
5280064,11012896,24213599,2,S22061A,10,"Stable burst fracture of T7-T8 vertebra, initi...",22.0,S22061A: Stable burst fracture of T7-T8 verteb...
5280065,11012896,24213599,4,M549,10,"Dorsalgia, unspecified",22.0,"M549: Dorsalgia, unspecified"
5280066,11012896,24213599,1,S22081A,10,"Stable burst fracture of T11-T12 vertebra, ini...",22.0,S22081A: Stable burst fracture of T11-T12 vert...
5280067,11012896,24213599,5,W109XXA,10,"Fall (on) (from) unspecified stairs and steps,...",22.0,W109XXA: Fall (on) (from) unspecified stairs a...


## Transaction encoding and association rule mining

The apriori function expects data in a one-hot encoded pandas DataFrame. Therefore we encode transactions with creating a crosstab from subject_id and icd_code_long_title with the help of pandas.

In [38]:
crosstab = pd.crosstab(diagnoses_depr_patients['subject_id'], diagnoses_depr_patients['icd_code_long_title'])
crosstab

icd_code_long_title,"A0471: Enterocolitis due to Clostridium difficile, recurrent","A0472: Enterocolitis due to Clostridium difficile, not specified as recurrent",A047: Enterocolitis due to Clostridium difficile,A071: Giardiasis [lambliasis],A0811: Acute gastroenteropathy due to Norwalk agent,"A084: Viral intestinal infection, unspecified","A09: Infectious gastroenteritis and colitis, unspecified","A1883: Tuberculosis of digestive tract organs, not elsewhere classified","A400: Sepsis due to streptococcus, group A",A4151: Sepsis due to Escherichia coli [E. coli],...,Z982: Presence of cerebrospinal fluid drainage device,Z9851: Tubal ligation status,Z9861: Coronary angioplasty status,Z9862: Peripheral vascular angioplasty status,Z9884: Bariatric surgery status,Z98890: Other specified postprocedural states,Z9889: Other specified postprocedural states,Z992: Dependence on renal dialysis,Z993: Dependence on wheelchair,Z9981: Dependence on supplemental oxygen
subject_id,,,,,,,,,,,,,,,,,,,,,
10002315,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10002528,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10002618,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10006630,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10007920,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19993501,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19996832,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19997887,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Check how many diagnoses a patient has and group them.

In [39]:
num_diagnoses = pd.DataFrame(crosstab[crosstab > 0].count(axis=1).sort_values(ascending=False), columns=['num_diagnoses'])
num_diagnoses['group'] = pd.cut(x=num_diagnoses['num_diagnoses'], bins=[0,1,2,3,4,5,9,19,29,49,1000], labels = ['1', '2', '3', '4', '5','6-9','10-19', '20-29', '30-49', '>=50'])
num_diagnoses_describe = num_diagnoses.groupby("group").describe()
num_diagnoses_describe.to_excel(r'output\num_diagnoses_experimental_group.xlsx')
num_diagnoses_describe

num_diagnoses                                                    
              count       mean       std   min   25%   50%    75%   max
group                                                                  
1              48.0   1.000000  0.000000   1.0   1.0   1.0   1.00   1.0
2             243.0   2.000000  0.000000   2.0   2.0   2.0   2.00   2.0
3             396.0   3.000000  0.000000   3.0   3.0   3.0   3.00   3.0
4             474.0   4.000000  0.000000   4.0   4.0   4.0   4.00   4.0
5             513.0   5.000000  0.000000   5.0   5.0   5.0   5.00   5.0
6-9          1533.0   7.303327  1.108934   6.0   6.0   7.0   8.00   9.0
10-19        1045.0  12.554067  2.522351  10.0  10.0  12.0  14.00  19.0
20-29         123.0  22.739837  2.798821  20.0  20.0  22.0  25.00  29.0
30-49          38.0  35.078947  4.564147  30.0  31.0  34.0  37.75  47.0
>=50            3.0  55.333333  3.055050  52.0  54.0  56.0  57.00  58.0

Binarize the transaction encoded dataset.

In [40]:
crosstab = crosstab.astype(bool)
crosstab

icd_code_long_title,"A0471: Enterocolitis due to Clostridium difficile, recurrent","A0472: Enterocolitis due to Clostridium difficile, not specified as recurrent",A047: Enterocolitis due to Clostridium difficile,A071: Giardiasis [lambliasis],A0811: Acute gastroenteropathy due to Norwalk agent,"A084: Viral intestinal infection, unspecified","A09: Infectious gastroenteritis and colitis, unspecified","A1883: Tuberculosis of digestive tract organs, not elsewhere classified","A400: Sepsis due to streptococcus, group A",A4151: Sepsis due to Escherichia coli [E. coli],...,Z982: Presence of cerebrospinal fluid drainage device,Z9851: Tubal ligation status,Z9861: Coronary angioplasty status,Z9862: Peripheral vascular angioplasty status,Z9884: Bariatric surgery status,Z98890: Other specified postprocedural states,Z9889: Other specified postprocedural states,Z992: Dependence on renal dialysis,Z993: Dependence on wheelchair,Z9981: Dependence on supplemental oxygen
subject_id,,,,,,,,,,,,,,,,,,,,,
10002315,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
10002528,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
10002618,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
10006630,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
10007920,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19993501,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
19996832,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
19997887,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


Generate frequent itemsets with mlxtend

In [41]:
frequent_itemsets = apriori(crosstab, min_support=0.08, use_colnames=True)
frequent_itemsets.sort_values(by=['support'], ascending=False).reset_index(drop=True)
frequent_itemsets_export = frequent_itemsets.copy()
frequent_itemsets_export['itemsets'] = frequent_itemsets_export['itemsets'].apply(lambda x: '; '.join(list(x))).astype("unicode")
frequent_itemsets_export

,support,itemsets
0,0.098958,"E785: Hyperlipidemia, unspecified"
1,0.193388,"F17210: Nicotine dependence, cigarettes, uncom..."
2,1.000000,"F329: Major depressive disorder, single episod..."
3,0.431159,"F419: Anxiety disorder, unspecified"
4,0.165308,"F4310: Post-traumatic stress disorder, unspeci..."
5,0.143116,I10: Essential (primary) hypertension
6,0.098279,"J45909: Unspecified asthma, uncomplicated"
7,0.118659,K219: Gastro-esophageal reflux disease without...
8,0.342165,R45851: Suicidal ideations
9,0.084239,Y929: Unspecified place or not applicable


Save frequent itemsets to Excel file.

In [42]:
frequent_itemsets_export.to_excel(r'output\frequent_itemsets_experimental_group.xlsx')

Generate association rules with mlxtend

In [43]:
assoc_rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.8)
assoc_rules.sort_values(by=['support'], ascending=False).reset_index(drop=True)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,"(F419: Anxiety disorder, unspecified)","(F329: Major depressive disorder, single episo...",0.431159,1.0,0.431159,1.0,1.0,0.0,inf
1,(R45851: Suicidal ideations),"(F329: Major depressive disorder, single episo...",0.342165,1.0,0.342165,1.0,1.0,0.0,inf
2,"(F17210: Nicotine dependence, cigarettes, unco...","(F329: Major depressive disorder, single episo...",0.193388,1.0,0.193388,1.0,1.0,0.0,inf
3,"(F4310: Post-traumatic stress disorder, unspec...","(F329: Major depressive disorder, single episo...",0.165308,1.0,0.165308,1.0,1.0,0.0,inf
4,"(F419: Anxiety disorder, unspecified, R45851: ...","(F329: Major depressive disorder, single episo...",0.158062,1.0,0.158062,1.0,1.0,0.0,inf
5,(I10: Essential (primary) hypertension),"(F329: Major depressive disorder, single episo...",0.143116,1.0,0.143116,1.0,1.0,0.0,inf
6,(Z915: Personal history of self-harm),"(F329: Major depressive disorder, single episo...",0.128170,1.0,0.128170,1.0,1.0,0.0,inf
7,(K219: Gastro-esophageal reflux disease withou...,"(F329: Major depressive disorder, single episo...",0.118659,1.0,0.118659,1.0,1.0,0.0,inf
8,"(E785: Hyperlipidemia, unspecified)","(F329: Major depressive disorder, single episo...",0.098958,1.0,0.098958,1.0,1.0,0.0,inf
9,"(J45909: Unspecified asthma, uncomplicated)","(F329: Major depressive disorder, single episo...",0.098279,1.0,0.098279,1.0,1.0,0.0,inf


In [44]:
assoc_rules['antecedents'] = assoc_rules['antecedents'].apply(lambda x: '; '.join(list(x))).astype("unicode")
assoc_rules['consequents'] = assoc_rules['consequents'].apply(lambda x: '; '.join(list(x))).astype("unicode")
assoc_rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,"E785: Hyperlipidemia, unspecified","F329: Major depressive disorder, single episod...",0.098958,1.0,0.098958,1.0,1.0,0.0,inf
1,"F17210: Nicotine dependence, cigarettes, uncom...","F329: Major depressive disorder, single episod...",0.193388,1.0,0.193388,1.0,1.0,0.0,inf
2,"F419: Anxiety disorder, unspecified","F329: Major depressive disorder, single episod...",0.431159,1.0,0.431159,1.0,1.0,0.0,inf
3,"F4310: Post-traumatic stress disorder, unspeci...","F329: Major depressive disorder, single episod...",0.165308,1.0,0.165308,1.0,1.0,0.0,inf
4,I10: Essential (primary) hypertension,"F329: Major depressive disorder, single episod...",0.143116,1.0,0.143116,1.0,1.0,0.0,inf
5,"J45909: Unspecified asthma, uncomplicated","F329: Major depressive disorder, single episod...",0.098279,1.0,0.098279,1.0,1.0,0.0,inf
6,K219: Gastro-esophageal reflux disease without...,"F329: Major depressive disorder, single episod...",0.118659,1.0,0.118659,1.0,1.0,0.0,inf
7,R45851: Suicidal ideations,"F329: Major depressive disorder, single episod...",0.342165,1.0,0.342165,1.0,1.0,0.0,inf
8,Y929: Unspecified place or not applicable,"F329: Major depressive disorder, single episod...",0.084239,1.0,0.084239,1.0,1.0,0.0,inf
9,Z590: Homelessness,"F329: Major depressive disorder, single episod...",0.090580,1.0,0.090580,1.0,1.0,0.0,inf


Export association rules to Excel file

In [45]:
assoc_rules.to_excel(r'output\association_rules_experimental_group.xlsx')

## Statistical analysis/Chi-squared test with control group

Randomly select control group of equal size as depression patients, with equal male/female and age group characteristics. Firstly, make sure that all control group patients have a diagnosis, because there are patients in the dataset that have no diagnoses.

In [46]:
patients_with_diagnoses = diagnoses['subject_id'].drop_duplicates().to_list()

In [47]:
patients = patients.copy()
patients = patients[patients.subject_id.isin(patients_with_diagnoses)]
not_depr_patients = patients[~patients.subject_id.isin(depr_patients_allseq)]
not_depr_patients = not_depr_patients.copy()
not_depr_patients['avg_age'] = not_depr_patients.subject_id.map(avg_age_dict)
not_depr_patients['age_group'] = pd.cut(x=not_depr_patients['avg_age'], bins=[-1,17,30,39,49,59,69,79,120], labels = ['0-17','18-29', '30-39', '40-49', '50-59', '60-69', '70-79', '>80'])

female = not_depr_patients[not_depr_patients.gender == 'F']
male = not_depr_patients[not_depr_patients.gender == 'M']

female18_29 = female.loc[female.age_group == '18-29', 'subject_id'].sample(n=1009, random_state=22).to_list()
female30_39 = female.loc[female.age_group == '30-39', 'subject_id'].sample(n=440, random_state=22).to_list()
female40_49 = female.loc[female.age_group == '40-49', 'subject_id'].sample(n=316, random_state=22).to_list()
female50_59 = female.loc[female.age_group == '50-59', 'subject_id'].sample(n=392, random_state=22).to_list()
female60_69 = female.loc[female.age_group == '60-69', 'subject_id'].sample(n=266, random_state=22).to_list()
female70_79 = female.loc[female.age_group == '70-79', 'subject_id'].sample(n=127, random_state=22).to_list()
female80 = female.loc[female.age_group == '>80', 'subject_id'].sample(n=60, random_state=22).to_list()


male18_29 = male.loc[male.age_group == '18-29', 'subject_id'].sample(n=648, random_state=22).to_list()
male30_39 = male.loc[male.age_group == '30-39', 'subject_id'].sample(n=330, random_state=22).to_list()
male40_49 = male.loc[male.age_group == '40-49', 'subject_id'].sample(n=249, random_state=22).to_list()
male50_59 = male.loc[male.age_group == '50-59', 'subject_id'].sample(n=307, random_state=22).to_list()
male60_69 = male.loc[male.age_group == '60-69', 'subject_id'].sample(n=195, random_state=22).to_list()
male70_79 = male.loc[male.age_group == '70-79', 'subject_id'].sample(n=54, random_state=22).to_list()
male80 = male.loc[male.age_group == '>80', 'subject_id'].sample(n=23, random_state=22).to_list()

control_grp = female18_29 + female30_39 + female40_49 + female50_59 + female60_69 + female70_79 + female80 + male18_29 + male30_39 + male40_49 + male50_59 + male60_69 + male70_79 + male80
len(control_grp)

4416

Create separate dataframe with control group patients and add demographic information.

In [48]:
control_grp_df = pd.DataFrame(control_grp, columns=['subject_id'])
control_grp_df['gender'] = control_grp_df.subject_id.map(gender_dict)
control_grp_df['age'] = control_grp_df.subject_id.map(avg_age_dict)
control_grp_df
control_grp_df['gender'].value_counts()

F    2610
M    1806
Name: gender, dtype: int64

In [49]:
control_grp_df['age_group'] = pd.cut(x=control_grp_df['age'], bins=[17,30,39,49,59,69,79,120], labels = ['18-29', '30-39', '40-49', '50-59', '60-69', '70-79', '>80'])
control_grp_df_describe = control_grp_df.groupby(["age_group"]).describe()
cols = [0,1,2,3,4,5,6,7]
control_grp_df_describe.drop(control_grp_df_describe.columns[cols],axis=1,inplace=True)
control_grp_df_describe.to_excel(r'output\age_control_group.xlsx')
control_grp_df_describe

age                                                   
            count       mean       std   min   25%   50%   75%   max
age_group                                                           
18-29      1657.0  24.820760  3.561042  18.0  22.0  25.0  28.0  30.0
30-39       770.0  34.761039  2.576283  31.0  32.0  35.0  37.0  39.0
40-49       565.0  44.812389  2.863352  40.0  42.0  45.0  47.0  49.0
50-59       699.0  55.107296  2.798115  50.0  53.0  55.0  57.0  59.0
60-69       461.0  64.681128  2.767091  60.0  62.0  65.0  67.0  69.0
70-79       181.0  74.099448  2.736797  70.0  72.0  74.0  76.0  79.0
>80          83.0  86.132530  4.023623  80.0  82.0  87.0  89.0  94.0

Include all diagnoses of control group patients.

In [50]:
control_grp_diagnoses = diagnoses[diagnoses['subject_id'].isin(control_grp)]

Create a new column as a combination of the icd_code and icd_code_long_title, which will help in interpretation of results.

In [51]:
control_grp_diagnoses = control_grp_diagnoses.copy()
control_grp_diagnoses['icd_code_long_title'] = control_grp_diagnoses['icd_code'].astype(str) + ": " + control_grp_diagnoses['long_title']
control_grp_diagnoses

,subject_id,hadm_id,seq_num,icd_code,icd_version,long_title,age,icd_code_long_title
2833780,17294018,27586051,4,Z3A40,10,40 weeks gestation of pregnancy,36.0,Z3A40: 40 weeks gestation of pregnancy
2833781,17294018,27586051,5,O701,10,Second degree perineal laceration during delivery,36.0,O701: Second degree perineal laceration during...
2833782,17294018,27586051,2,Z370,10,Single live birth,36.0,Z370: Single live birth
2833783,17294018,27586051,1,O76,10,Abnormality in fetal heart rate and rhythm com...,36.0,O76: Abnormality in fetal heart rate and rhyth...
2833784,17294018,27586051,3,O6981X0,10,Labor and delivery complicated by cord around ...,36.0,O6981X0: Labor and delivery complicated by cor...
...,...,...,...,...,...,...,...,...
5280115,19568216,22381165,5,I252,10,Old myocardial infarction,49.0,I252: Old myocardial infarction
5280116,19568216,22381165,3,I2510,10,Atherosclerotic heart disease of native corona...,49.0,I2510: Atherosclerotic heart disease of native...
5280117,19568216,22381165,1,R079,10,"Chest pain, unspecified",49.0,"R079: Chest pain, unspecified"
5280118,19568216,22381165,8,E7800,10,"Pure hypercholesterolemia, unspecified",49.0,"E7800: Pure hypercholesterolemia, unspecified"


Create crosstab for control group with subject_id and icd_code_long_title. 

In [52]:
control_grp_crosstab = pd.crosstab(control_grp_diagnoses['subject_id'], control_grp_diagnoses['icd_code_long_title'])

Check how many diagnoses a patient in the control group has and group them.

In [53]:
num_diagnoses_control_grp = pd.DataFrame(control_grp_crosstab[control_grp_crosstab > 0].count(axis=1).sort_values(ascending=False), columns=['num_diagnoses'])
num_diagnoses_control_grp['group'] = pd.cut(x=num_diagnoses_control_grp['num_diagnoses'], bins=[0,1,2,3,4,5,9,19,29,49,1000], labels = ['1', '2', '3', '4', '5','6-9','10-19', '20-29', '30-49', '>=50'])
num_diagnoses_control_grp_describe = num_diagnoses_control_grp.groupby("group").describe()
num_diagnoses_control_grp_describe.to_excel(r'output\num_diagnoses_control_group.xlsx')
num_diagnoses_control_grp_describe

num_diagnoses                                                     
              count       mean        std   min   25%   50%   75%    max
group                                                                   
1             162.0   1.000000   0.000000   1.0   1.0   1.0   1.0    1.0
2             259.0   2.000000   0.000000   2.0   2.0   2.0   2.0    2.0
3             331.0   3.000000   0.000000   3.0   3.0   3.0   3.0    3.0
4             362.0   4.000000   0.000000   4.0   4.0   4.0   4.0    4.0
5             368.0   5.000000   0.000000   5.0   5.0   5.0   5.0    5.0
6-9          1160.0   7.387931   1.101192   6.0   6.0   7.0   8.0    9.0
10-19        1121.0  13.432649   2.798144  10.0  11.0  13.0  16.0   19.0
20-29         332.0  23.900602   2.853799  20.0  21.0  24.0  26.0   29.0
30-49         228.0  37.486842   5.632255  30.0  33.0  37.0  42.0   49.0
>=50           93.0  67.473118  19.502280  50.0  54.0  60.0  76.0  168.0

Binarize the dataframe.

In [54]:
control_grp_crosstab = control_grp_crosstab.astype(bool)
control_grp_crosstab

icd_code_long_title,A020: Salmonella enteritis,"A029: Salmonella infection, unspecified","A039: Shigellosis, unspecified",A044: Other intestinal Escherichia coli infections,A045: Campylobacter enteritis,"A0471: Enterocolitis due to Clostridium difficile, recurrent","A0472: Enterocolitis due to Clostridium difficile, not specified as recurrent",A047: Enterocolitis due to Clostridium difficile,A048: Other specified bacterial intestinal infections,"A049: Bacterial intestinal infection, unspecified",...,Z9851: Tubal ligation status,Z9861: Coronary angioplasty status,Z9884: Bariatric surgery status,Z98890: Other specified postprocedural states,Z98891: History of uterine scar from previous surgery,Z9889: Other specified postprocedural states,Z9911: Dependence on respirator [ventilator] status,Z992: Dependence on renal dialysis,Z993: Dependence on wheelchair,Z9981: Dependence on supplemental oxygen
subject_id,,,,,,,,,,,,,,,,,,,,,
10002545,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
10003637,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
10010663,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
10010993,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
10015132,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19991111,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
19992418,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
19994717,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [55]:
frequent_itemsets_control_grp = apriori(control_grp_crosstab, min_support=0.001, use_colnames=True, low_memory=True)
frequent_itemsets_control_grp.sort_values(by=['support'], ascending=False).reset_index(drop=True)
frequent_itemsets_control_grp.to_excel(r'output\frequent_itemsets_control_grp.xlsx')